In [80]:
from bs4 import BeautifulSoup
import numpy as np
import sqlite3
from datetime import datetime 
import requests
import pandas as pd


In [81]:
URL ='https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks'
attributes =['Name','MC_USD_Billion']
csv_path = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-PY0221EN-Coursera/labs/v2/exchange_rate.csv'
db_name = 'Banks.db'
output_path = './Largest_banks_data.csv'
table_name='Largest_banks'
log_file = 'code_log.txt'


# Task 1

In [82]:
def log_progress(message):
    timestamp_format = '%Y-%h-%d-%H:%M:%S'
    now = datetime.now()
    timestamp = now.strftime(timestamp_format) 
    with open("./code_log.txt","a") as f: 
        f.write(timestamp + ' : ' + message + '\n')  

# Task 2

In [83]:
def extract(url, table_attribs):
    page = requests.get(url).text
    data = BeautifulSoup(page, 'html.parser')
    df = pd.DataFrame(columns=table_attribs)
    
    tables = data.find_all('tbody')
    rows = tables[0].find_all('tr')
    
    for row in rows:
        col = row.find_all('td')
        if len(col) != 0:
            if col[1].find('a') is not None:
                data_cell = col[1].find_all('a')
                if len(data_cell) >= 2:
                    bank_name = data_cell[1]['title']
                else:
                    bank_name = data_cell[0].contents[0]
                
                market_cap = col[2].contents[0][:-1]
                
                data_dict = {
                    "Name": bank_name,
                    "MC_USD_Billion": float(market_cap)
                }
                df1 = pd.DataFrame(data_dict, index=[0])
                df = pd.concat([df, df1], ignore_index=True)
    
    return df


In [84]:
df = extract(URL,attributes)
df

C:\Users\ancir\AppData\Local\Temp\ipykernel_20184\2723134287.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, df1], ignore_index=True)


,Name,MC_USD_Billion
0,JPMorgan Chase,432.92
1,Bank of America,231.52
2,Industrial and Commercial Bank of China,194.56
3,Agricultural Bank of China,160.68
4,HDFC Bank,157.91
5,Wells Fargo,155.87
6,HSBC,148.90
7,Morgan Stanley,140.83
8,China Construction Bank,139.82
9,Bank of China,136.81


# Task 3

In [85]:
def transform(df, csv_path):
    
    exchange_rate_df = pd.read_csv(csv_path)
    exchange_rate = exchange_rate_df.set_index('Currency').to_dict()['Rate']
    
    df['MC_GBP_Billion'] = [np.round(float(x) * float(exchange_rate['GBP']), 2) for x in df['MC_USD_Billion']]
    df['MC_EUR_Billion'] = [np.round(float(x) * float(exchange_rate['EUR']), 2) for x in df['MC_USD_Billion']]
    df['MC_INR_Billion'] = [np.round(float(x) * float(exchange_rate['INR']), 2) for x in df['MC_USD_Billion']]
    
    return df


In [86]:
df = transform(df,csv_path)
df

,Name,MC_USD_Billion,MC_GBP_Billion,MC_EUR_Billion,MC_INR_Billion
0,JPMorgan Chase,432.92,346.34,402.62,35910.71
1,Bank of America,231.52,185.22,215.31,19204.58
2,Industrial and Commercial Bank of China,194.56,155.65,180.94,16138.75
3,Agricultural Bank of China,160.68,128.54,149.43,13328.41
4,HDFC Bank,157.91,126.33,146.86,13098.63
5,Wells Fargo,155.87,124.70,144.96,12929.42
6,HSBC,148.90,119.12,138.48,12351.26
7,Morgan Stanley,140.83,112.66,130.97,11681.85
8,China Construction Bank,139.82,111.86,130.03,11598.07
9,Bank of China,136.81,109.45,127.23,11348.39


# Task 4

In [87]:
def load_to_csv(df, output_path):
    df.to_csv(output_path)

In [88]:
load_to_csv(df,output_path)

# Task 5

In [89]:
def load_to_db(df, sql_connection, table_name):
    df.to_sql(table_name, sql_connection, if_exists='replace', index=False)

In [90]:
conn = sqlite3.connect(db_name)
load_to_db(df, conn, table_name)

# Task 6

In [91]:
def run_query(query_statement, sql_connection):
    print(query_statement)
    query_output = pd.read_sql(query_statement, sql_connection)
    print(query_output)

In [92]:
query_statement_1 = f"SELECT * FROM {table_name}"
run_query(query_statement_1, conn)
query_statement_2 = f"SELECT AVG(MC_GBP_Billion) FROM {table_name}"
run_query(query_statement_2, conn)
query_statement_3 = f"SELECT Name from {table_name} LIMIT 5"
run_query(query_statement_3, conn)


SELECT * FROM Largest_banks
                                      Name  MC_USD_Billion  MC_GBP_Billion  \
0                           JPMorgan Chase          432.92          346.34   
1                          Bank of America          231.52          185.22   
2  Industrial and Commercial Bank of China          194.56          155.65   
3               Agricultural Bank of China          160.68          128.54   
4                                HDFC Bank          157.91          126.33   
5                              Wells Fargo          155.87          124.70   
6                                     HSBC          148.90          119.12   
7                           Morgan Stanley          140.83          112.66   
8                  China Construction Bank          139.82          111.86   
9                            Bank of China          136.81          109.45   

   MC_EUR_Billion  MC_INR_Billion  
0          402.62        35910.71  
1          215.31        19204.58  
2    

# Task 7

In [93]:
from datetime import datetime

log_file = 'code_log.txt'
timestamp_format = '%Y-%h-%d-%H:%M:%S'

with open(log_file, "w") as f:
    f.write(datetime.now().strftime(timestamp_format) + " : Preliminaries complete. Initiating ETL process\n")
    f.write(datetime.now().strftime(timestamp_format) + " : Data extraction complete. Initiating Transformation process\n")
    f.write(datetime.now().strftime(timestamp_format) + " : Data transformation complete. Initiating Loading process\n")
    f.write(datetime.now().strftime(timestamp_format) + " : Data saved to CSV file\n")
    f.write(datetime.now().strftime(timestamp_format) + " : SQL Connection initiated\n")
    f.write(datetime.now().strftime(timestamp_format) + " : Data loaded to Database as a table, Executing queries\n")
    f.write(datetime.now().strftime(timestamp_format) + " : Process Complete\n")
    f.write(datetime.now().strftime(timestamp_format) + " : Server Connection closed\n")
